In [8]:
import os
import boto3
import json
import pandas as pd

In [9]:
os.environ["AWS_ACCESS_KEY_ID"] = "dummy"
os.environ["AWS_SECRET_ACCESS_KEY"] = "dummy"
os.environ["AWS_ENDPOINT_URL"] = "http://localhost:4566"
os.environ["AWS_REGION"] = "ap-south-1"

In [13]:
# Import necessary libraries
from pyflink.table import EnvironmentSettings, TableEnvironment
import os

# 1. Create a Table Environment
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)

# Define configuration properties for the Flink job
props = [
    {
        "PropertyGroupId": "kinesis.analytics.flink.run.options",
        "PropertyMap": {
            "python": "GettingStarted/getting-started.py",  # Python script to be executed
            "jarfile": "flink-sql-connector-kinesis-1.16.1.jar"  # Flink connector JAR file
        }
    },
    {
        "PropertyGroupId": "consumer.config.0",
        "PropertyMap": {
            "input.stream.name": "input-stream",        # Name of the Kinesis Data Stream
            "flink.stream.initpos": "TRIM_HORIZON",     # Initial position to start reading the stream
            "aws.region": "ap-south-1"                   # AWS region where the stream is located
        }
    }
]

# Function to extract property map from the configuration properties
def property_map(props, property_group_id):
    for prop in props:
        if prop["PropertyGroupId"] == property_group_id:
            return prop["PropertyMap"]

# Function to create a source table definition for Kinesis stream
# TODO windowing on this create table statement to store the states
def create_source_table(table_name, stream_name, region, stream_initpos):
    return f""" 
    CREATE TABLE {table_name} (
        event_uuid VARCHAR,
        event_name VARCHAR,
        event_data VARCHAR,
        event_type VARCHAR,
        event_timestamp INTEGER,
        shard_id INTEGER,
        event_source VARCHAR,
        event_version VARCHAR
    )
    PARTITIONED BY (event_name)
    WITH (
        'connector' = 'kinesis',
        'stream' = '{stream_name}',
        'aws.region' = '{region}',
        'aws.endpoint' = 'http://localhost:4566',
        'aws.credentials.basic.accesskeyid' = 'dummy',
        'aws.credentials.basic.secretkey' = 'dummy',
        'scan.stream.initpos' = '{stream_initpos}',
        'format' = 'json',
        'json.timestamp-format.standard' = 'ISO-8601'
    ) 
    """

# Main function
def main():
    # Define keys for accessing properties
    input_property_group_key = "consumer.config.0"
    input_stream_key = "input.stream.name"
    input_region_key = "aws.region"
    input_starting_position_key = "flink.stream.initpos"

    # Table name for the source data
    input_table_name = "input_table"

    # Get application properties from the props list
    input_property_map = property_map(props, input_property_group_key)
    input_stream = input_property_map[input_stream_key]
    input_region = input_property_map[input_region_key]
    stream_initpos = input_property_map[input_starting_position_key]

    # Check if running in a local environment and set pipeline.jars configuration
    CURRENT_DIR = os.getcwd()  # Get the current working directory
    table_env.get_config().get_configuration().set_string(
        "pipeline.jars",
        "file:///" + CURRENT_DIR + "/flink-sql-connector-kinesis-1.16.1.jar",
    )

    # 2. Create a source table from a Kinesis Data Stream
    table_env.execute_sql(
        create_source_table(input_table_name, input_stream, input_region, stream_initpos)
    )

    print("Successfully created source tables ")

    # 3. Read data from the source table and print it to the console
    query = f"SELECT * FROM {input_table_name}"
    print(f"Executing query: {query}")
    
    # table_env.execute_sql(query).print()
    


# Entry point for the script
if __name__ == "__main__":
    main()

Successfully created source tables 
Executing query: SELECT * FROM input_table


In [14]:
# table_env.execute_sql("show tables " ).print()

In [15]:
# table_env.execute_sql("select * from  input_table " ).print()

In [16]:
# add required jar files

CURRENT_DIR = os.getcwd()

# Define a list of JAR file names you want to add
jar_files = [
    "flink-s3-fs-hadoop-1.16.1.jar",
    "hudi-flink1.13-bundle-0.13.1.jar",
    "flink-sql-connector-kinesis-1.16.1.jar"
]

# Build the list of JAR URLs by prepending 'file:///' to each file name
jar_urls = [f"file:///{CURRENT_DIR}/{jar_file}" for jar_file in jar_files]
      
table_env.get_config().get_configuration().set_string(
    "pipeline.jars",
    ";".join(jar_urls)
)

In [17]:
# write to hudi 
s3_bucket = 'south-test-1'
hudi_output_path = f's3a://{s3_bucket}/tmp1'

hudi_sink = f"""
DROP TABLE IF EXISTS hand_data;

CREATE TABLE hand_data(
    event_uuid VARCHAR,
    event_name VARCHAR,
    event_data VARCHAR,
    event_type VARCHAR,
    event_timestamp INTEGER,
    shard_id INTEGER,
    event_source VARCHAR,
    event_version VARCHAR
)
PARTITIONED BY (`event_name`)
WITH (
    'connector' = 'hudi',
    'path' = '{hudi_output_path}' ,
    'table.type' = 'MERGE_ON_READ' ,
    'read.streaming.enabled' = 'true'
);
"""

print(f"{hudi_sink}")

# Execute the SQL to create the Hudi table
# table_env.execute_sql(hudi_sink)


# Execute the DDL statements
for statement in hudi_sink.strip().split(';'):
    if statement.strip():
        table_env.execute_sql(statement.strip())

# Define the data to be inserted into the Hudi table
table_env.execute_sql("""
    INSERT INTO hand_data
        SELECT * FROM input_table
""").print()


DROP TABLE IF EXISTS hand_data;

CREATE TABLE hand_data(
    event_uuid VARCHAR,
    event_name VARCHAR,
    event_data VARCHAR,
    event_type VARCHAR,
    event_timestamp INTEGER,
    shard_id INTEGER,
    event_source VARCHAR,
    event_version VARCHAR
)
PARTITIONED BY (`event_name`)
WITH (
    'connector' = 'hudi',
    'path' = 's3a://south-test-1/tmp1' ,
    'table.type' = 'MERGE_ON_READ' ,
    'read.streaming.enabled' = 'true'
);



Py4JJavaError: An error occurred while calling o332.executeSql.
: org.apache.flink.table.api.ValidationException: Unable to create a sink for writing table 'default_catalog.default_database.hand_data'.

Table options are:

'connector'='hudi'
'path'='s3a://south-test-1/tmp1'
'read.streaming.enabled'='true'
'table.type'='MERGE_ON_READ'
	at org.apache.flink.table.factories.FactoryUtil.createDynamicTableSink(FactoryUtil.java:331)
	at org.apache.flink.table.planner.delegation.PlannerBase.getTableSink(PlannerBase.scala:451)
	at org.apache.flink.table.planner.delegation.PlannerBase.translateToRel(PlannerBase.scala:227)
	at org.apache.flink.table.planner.delegation.PlannerBase.$anonfun$translate$1(PlannerBase.scala:177)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:233)
	at scala.collection.Iterator.foreach(Iterator.scala:937)
	at scala.collection.Iterator.foreach$(Iterator.scala:937)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1425)
	at scala.colle

In [43]:
table_env.execute_sql("""
    show tables
""").print()

+-------------+
|  table name |
+-------------+
|   hand_data |
| input_table |
+-------------+
2 rows in set


In [7]:
table_env.execute_sql("""
    drop table input_table
""").print()

OK
